In [68]:
%load_ext autoreload
%autoreload 2

#Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sympy import symbols, diff, lambdify
from scipy.optimize import minimize

#Py files
import multijoueurs_Q as multiQ
import resultsPriceandProfits as res 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
prices = res.PriceOptimizer(nb_players=2)
collusion_profit, collusion_price = prices.CollusionPrice()
nash_profit, nash_price = prices.NashPrice()

print("Collusion profit:", collusion_profit, "Collusion price:", collusion_price)
print("Nash profit:",nash_profit, "Nash price:",nash_price)

In [118]:
prices = res.PriceOptimizer(binary_demand=True)
collusion_profit, collusion_price = prices.CollusionPrice()
nash_profit, nash_price = prices.NashPrice()

print("Collusion profit:", collusion_profit, "Collusion price:", collusion_price)
print("Nash profit:",nash_profit, "Nash price:",nash_price)

Collusion profit: 0.125 Collusion price: [1.5]
Nash profit: 0.0 Nash price: [1. 1.]


In [116]:
import numpy as np
 # Changer le nombre de joueurs ici

aggregated_agents = []
for demand in [True, False]:
    total_rewards = []

    for loop in range(3):
        print("Loop:", loop)
        agents = [multiQ.Agent(binary_demand=demand) for _ in range(2)]
        env = multiQ.Env(binary_demand=demand)

        temps = []
        rewards = []
        epsilon = []
        prices = []

        # Initialisation des prix p0 (on va le faire directement dans chaque agent)
        for agent in agents:
            agent.p = np.random.choice(agent.A)

        # Initialisation de l'état
        s_t = env([agent.p for agent in agents])[1]
        for agent in agents:
            agent.s_t = s_t

        s_ind = agents[0].find_index(agents[0].S, agents[0].s_t)
        for agent in agents:
            agent.s_ind = s_ind

        # Phase itérative
        for t in range(10**4):
            # Actions et état t+1
            for agent in agents:
                agent.a_ind = agent.get_next_action()

            s_t1 = env([agent.A[agent.a_ind] for agent in agents])[1]
            for agent in agents:
                agent.s_t1 = s_t1

            s_ind1 = agents[0].find_index(agents[0].S, agents[0].s_t1)
            for agent in agents:
                agent.s_ind1 = s_ind1

            temps.append(t)
            ret = env(s_t1)
            quant, price, cost = ret

            re = ret[0]*ret[1]-ret[0]*ret[2]
            rewards.append(re)
            epsilon_values = [agent.epsilon for agent in agents]
            epsilon.append(epsilon_values)
            prices.append([agent.p for agent in agents])

            for i, agent in enumerate(agents):
                agent.updateQ(q=quant[i], p=price[i], c=cost[i], t=t)

        total_rewards.append(rewards)

    aggregated_agents.append(np.array(total_rewards).mean(axis=0))

Loop: 0
Loop: 1
Loop: 2
Loop: 0
Loop: 1
Loop: 2


In [117]:
aggregated_agents

[array([[ 0.09316327,  0.01147959],
        [ 0.03125   ,  0.16727041],
        [-0.0175    ,  0.0805102 ],
        ...,
        [ 0.02415816,  0.0392602 ],
        [ 0.02413265,  0.14336735],
        [ 0.0057398 ,  0.10625   ]]),
 array([[0.28770967, 0.24139762],
        [0.26320309, 0.34184172],
        [0.25271541, 0.33527759],
        ...,
        [0.2163537 , 0.28421879],
        [0.31148833, 0.32510304],
        [0.33788134, 0.24178372]])]

In [124]:
prices = res.PriceOptimizer(nb_players=2)
collusion_profit, collusion_prices = prices.CollusionPrice()
nash_profit, nash_prices = prices.NashPrice()

prices_binary = res.PriceOptimizer(nb_players=2, binary_demand=True)
collusion_profit_binary, collusion_prices_binary = prices_binary.CollusionPrice()
nash_profit_binary, nash_prices_binary = prices_binary.NashPrice()

RN = [nash_profit_binary, nash_profit]
RC = [collusion_profit_binary, collusion_profit]
    
Rmean = np.zeros((len(aggregated_agents), 2))
DRmean = np.zeros((len(aggregated_agents), 2))   
    
for i in range (len(aggregated_agents)): 
    for j in range (2):
        Rmean[i][j] = aggregated_agents[i][-100:,j].mean()
        DRmean[i][j] = (Rmean[i][j] - RN[i])/(RC[i] - RN[i]) 

print(RN, RC)
print(Rmean)
print(DRmean)

av = np.mean(DRmean, axis=1)
print(av)

print("Mean $\\Delta$ for binary demand:", av[0])
print("Mean $\\Delta$ for 'exponential' demand:", av[1])

[0.0, 0.2216948057185367] [0.125, 0.3374904593213026]
[[0.05156811 0.04831556]
 [0.28387555 0.27171283]]
[[0.4125449  0.38652449]
 [0.53698686 0.43195078]]
[0.39953469 0.48446882]
Mean $\Delta$ for binary demand: 0.3995346938775508
Mean $\Delta$ for 'exponential' demand: 0.48446881671395725
